In [1]:
# postgres passwd
passwd = "arst"


In [1]:
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.svm import SVC
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from queries import *
from model_test import ModelTest

# We are also going to do some basic viz
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline

!dropdb chicago_crimes
!createdb chicago_crimes

In [ ]:
BaggingClassifier()

In [ ]:
SVC()

In [3]:
connection_args = {
    'host': '127.0.0.1', # You'll have to update this to your IP
    'user': 'john',    # username
    'password': passwd,
    'dbname': 'chicago_crimes',
    'port': 5432}
connection = pg.connect(**connection_args)
cursor = connection.cursor()

## basic cleaning

cwd = !pwd
cwd = cwd[0]
csvs = !ls data/Chicago_Crimes_20*.csv
#!chmod a+rX ../.. .. . ./data ./data/*

#!chmod a+rX ../.. .. . ./data ./data/*
#!sed -i '/5302454,HM711750/d' data/Chicago_Crimes_2005_to_2007.csv
#!sed -i '/08\/17\/2015 03:03:40 PM,,,/d' data/Chicago_Crimes_2008_to_2011.csv

cursor.execute(build_tables_sql)
for filename in csvs:
    filepath = cwd + "/" + filename
    print(filepath)
    query = load_csv_str.format(filepath)
    cursor.execute(query)
cursor.execute("COMMIT;")

### Data retrieval

In [6]:
target = 'primary_type'
features_sql = ", ".join([
     "latitude"
    , "longitude"
    , "datetime"
    , "domestic"
    #, "location_desc"
    ])

query = "SELECT {} FROM crimes WHERE latitude > 36.62 AND datetime > '2015-01-01'"
ys = pd_sql.read_sql(query.format(target), connection)
Xs = pd_sql.read_sql(query.format(features_sql), connection)

query = "SELECT primary_type, count(*) FROM crimes GROUP BY primary_type ORDER BY count DESC"
pd_sql.read_sql(query, connection)

### Modelling

In [6]:
ssc = StandardScaler()
Xs = ssc.fit_transform(Xs)
data_tuple = train_test_split(Xs, ys, random_state=4)

In [7]:
dummies = 

depths = range(15,25,1)
trees = [RandomForestClassifier(n_estimators=250, max_depth=depth, n_jobs = 3 ) for depth in depths] 

knn_neighbors = range(80,150,9)
knns =  [KNeighborsClassifier(n_neighbors=n, metric="manhattan", n_jobs=3)
            for n in knn_neighbors]

logs = [LogisticRegression(n_jobs=3, dual=False),
        LogisticRegression(n_jobs=3, solver="sag"),
        LogisticRegression(n_jobs=3, solver="sag", tol=0.1)]
models = trees + knns + logs

In [ ]:
last_record = !tail model_results.csv -n 1
last_record_list = str(last_record).split(",")
last_id_str = last_record_list[0][2:]
last_id = int(last_id_str)

In [ ]:
for i, m in enumerate(models):
    test = ModelTest(m, *data_tuple, id_ = last_id + 1 +i)
    test.log()

In [ ]:
wtf_q = """
    SELECT * FROM crimes 
    WHERE latitude < 36.62 OR longitude < -91.6
    """

wtf_df = pd_sql.read_sql(wtf_q, connection)
wtf_df